In [2]:
# file to mix pure state rhos into mixed states and save them as csvs
import numpy as np
from os.path import join, dirname, abspath
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import random

from uncertainties import ufloat
from uncertainties import unumpy as unp

from sample_rho import *
from rho_methods import *
from process_expt_lev import *

DATA_PATH = 'mixed_phi_psi_45'
 # DEF GET THEO RHO

# DEF MIX RHO
    
def mix_states(file_names, probs, state_name):
    '''
    This returns mixed rhos and saves a csv with a mixed state.
    Note that uncertainity is taken as a mean, as is purity and fidelity.
    This file assumes one chi is being analyzed at a time.
    
    Parameters:
    file_names (list of lists): For N files, N lists of lists of the file names of each file,
                                organized in order of chi.
    probs (list): probability of the rho in each file.
    state_name (str): name of state to be used when saving data
    
    Returns:
    rhos (list): list of merged rhos
    '''
    
    # Obtain the data from each file, and use probs: 
    row_names = ['rho', 'unc', 'Su', 'un_proj', 'un_proj_unc', '_', 'angles', 'fidelity', 'purity']
    df = pd.DataFrame(index=row_names, columns=file_names) 
    for i, file in enumerate(file_names):
        data = np.load(join(DATA_PATH, file), allow_pickle=True)
        for j, row_name in enumerate(row_names):
            df.loc[row_name, file] = data[j]
        # Use probabilities on all values that I felt like
        df.loc['rho', file] = probs[i] * df.loc['rho', file]
        df.loc['unc', file] = probs[i] * df.loc['unc', file]
        df.loc['un_proj', file] = probs[i] * df.loc['un_proj', file]
        df.loc['un_proj_unc', file] = probs[i] * df.loc['un_proj_unc', file]

    # Create a new data frame and propagate it with previous data frames values
    df_to_save = pd.DataFrame()
    df_to_save.index = row_names
    df_to_save['col'] = df[file_names[0]]
    for i, file_name in enumerate(file_names):
        if file_name == file_names[0]:  # Skip the first bc that is already in the new df
            pass
        else:
            df_to_save['col'] += df[file_name]
            
    # normalize the angle, purity, and fidelity values
    df_to_save.loc['angles', 'col'] = [x / len(file_names) for x in df_to_save.loc['angles', 'col']]
    df_to_save.loc['purity', 'col'] /= len(file_names)
    df_to_save.loc['fidelity', 'col'] /= len(file_names)
    print(df_to_save.head())
    np.save(join(DATA_PATH,f"rho_('E0', {state_name})_mix"), df_to_save.values)
    
    """
    NEXT STEPS:
    - create mixed_rho_gen function within process_expt_lev
    - create a function that uses mixed_rho to use with various chi (may just be main = init?)
    """    

In [18]:
# define experimental parameters
etas = [np.pi/4]
chis = np.linspace(0.001, np.pi/2, 6)
probs = [0.65, 0.35]
states_names = []
states = []

# define state names
for eta in etas:
    for chi in chis:
        states_names.append((np.rad2deg(eta), np.rad2deg(chi)))
        states.append((eta, chi))

# get new csvs of mixed state
for i, state_n in enumerate(states_names):
    filenames = []
    filenames.append(f"rho_('E0', {state_n})_1.npy")
    filenames.append(f"rho_('E0', {state_n})_26.npy")
    mix_states(filenames, probs, state_n)
    row_names = ['rho', 'unc', 'Su', 'un_proj', 'un_proj_unc', '_', 'angles', 'fidelity', 'purity']
    df = pd.DataFrame(index=row_names, columns=filenames) 
    for i, file in enumerate(filenames):
        data = np.load(join(DATA_PATH, file), allow_pickle=True)
        for j, row_name in enumerate(row_names):
            df.loc[row_name, file] = data[j]
        # Use probabilities on all values that I felt like
        df.loc['rho', file] = probs[i] * df.loc['rho', file]
        df.loc['unc', file] = probs[i] * df.loc['unc', file]
        df.loc['un_proj', file] = probs[i] * df.loc['un_proj', file]
        df.loc['un_proj_unc', file] = probs[i] * df.loc['un_proj_unc', file]
        df.loc['purity', file] = probs[i] * df.loc['purity', file] 
        df.loc['fidelity', file] = probs[i] * df.loc['fidelity', file]
    # Create a new data frame and propagate it with previous data frames values
    df_to_save = pd.DataFrame()
    df_to_save.index = row_names
    df_to_save['col'] = df[filenames[0]]
    for i, file_name in enumerate(filenames):
        if file_name == filenames[0]:  # Skip the first bc that is already in the new df
            pass
        else:
            df_to_save['col'] += df[file_name]
            
    # normalize the angle values
    df_to_save.loc['angles', 'col'] = [x / len(filenames) for x in df_to_save.loc['angles', 'col']]
    
    np.save(join(DATA_PATH,f"rho_('E0', {state_n})_mix"), df_to_save.values)

                                                           col
rho          [[(0.64425081352554+0j), (0.022826221983076562...
unc          [[(0.0022626403316052326+0j), (0.0009647169668...
Su           [[0.0, 0.01261154571768091, 0.0151786956198170...
un_proj      [[1838.6299999999999, 983.06, 1492.19, 1364.5,...
un_proj_unc  [[11.250621789771785, 5.102658800025761, 18.46...
                                                           col
rho          [[(0.6294214630261193+0j), (0.0233317337754949...
unc          [[(0.005005601923101759+0j), (0.00085472775512...
Su           [[0.0, 0.01688649600542897, 0.0191633334252606...
un_proj      [[1769.93, 998.5699999999998, 1478.81, 1337.88...
un_proj_unc  [[24.059116831522026, 9.316756859362, 19.14351...
                                                           col
rho          [[(0.5820405856228961+0j), (0.0199452829858771...
unc          [[(0.00407123508877181+0j), (0.000704592092524...
Su           [[0.0, 0.014273646646226702, 0.01615829406

In [19]:
df.head(10)

,"rho_('E0', (45.0, 90.0))_1.npy","rho_('E0', (45.0, 90.0))_26.npy"
rho,"[[(0.32197409256510595+0j), (0.008169824831072...","[[(0.0012034649209812717+0j), (-0.001422915787..."
unc,"[[(0.0018302202377616357+0j), 0.00078049868638...","[[(0.0008303607528174462+0j), 0.00052204149108..."
Su,"[[0.0, 0.0057648710685198395, 0.00816327591156...","[[0.0, 0.008654740130094011, 0.010671441146374..."
un_proj,"[[954.46, 7.41, 513.37, 434.71999999999997, 45...","[[3.6399999999999997, 521.5699999999999, 263.2..."
un_proj_unc,"[[9.378038174373145, 0.39, 6.916928509099975, ...","[[0.4509434554353793, 4.501821853427787, 5.484..."
_,"(0.7853981633974483, 1.5707963267948966)","(0.7853981633974483, 1.5707963267948966)"
angles,"[-111.02239096250102, -14.2599139506072, 0]","[62.28602543604673, 329.18026785714284, 45]"
fidelity,0.632003,0.340212
purity,0.619501,0.331132


In [20]:
df_to_save.head(10)

,col
rho,"[[(0.32317755748608723+0j), (0.006746909043789..."
unc,"[[(0.002660580990579082+0j), 0.001302540177463..."
Su,"[[0.0, 0.01441961119861385, 0.0188347170579353..."
un_proj,"[[958.1, 528.9799999999999, 776.64, 698.27, 71..."
un_proj_unc,"[[9.828981629808524, 4.891821853427786, 12.401..."
_,"(0.7853981633974483, 1.5707963267948966, 0.785..."
angles,"[-55.51119548125051, -7.1299569753036, 0.0, 31..."
fidelity,0.972215
purity,0.950633
